In [14]:
from ultralytics import YOLO

import matplotlib.pyplot as plt 
import seaborn as sns 
import os 
import cv2
import numpy as np

In [17]:
model_path = "./segment/train/weights/best.pt"

image_path = "./datasets/all/duck_0.jpeg"

img = cv2.imread(image_path)
H, W, _ = img.shape

model = YOLO(model_path)
results = model(img)

for result in results:
    for j, mask in enumerate(result.masks.data):
        # print(mask)
        mask = mask.numpy() * 255
        mask = cv2.resize(mask, (W, H))
        cv2.imwrite('./output.png', mask)


0: 448x640 1 duck, 248.9ms
Speed: 7.7ms preprocess, 248.9ms inference, 8.3ms postprocess per image at shape (1, 3, 448, 640)


In [26]:
import cv2

def combine_images(img, mask):
    # Resize mask if necessary
    if mask.shape[:2] != img.shape[:2]:
        mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

    # Convert mask to RGB if necessary
    if len(mask.shape) == 2:
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # Add the images with weights
    output = cv2.addWeighted(img, 0.8, mask, 0.2, 0)

    return output

img = cv2.imread(image_path)
mask = cv2.imread("./output.png")

output = combine_images(img, mask)

cv2.imwrite("./combined.png", output)


True

In [31]:
import cv2
import numpy as np
from ultralytics import YOLO  # Pastikan modul YOLO Anda diimport dengan benar

model_path = "./segment/train/weights/best.pt"
# image_path = "./datasets/all/duck_0.jpeg"
image_path = "./testing/bebek_putih.webp"

img = cv2.imread(image_path)
H, W, _ = img.shape

model = YOLO(model_path)
results = model(img)

# Membuat array kosong untuk menyimpan semua masker
all_masks = []

for result in results:
    for j, mask in enumerate(result.masks.data):
        mask = mask.numpy() * 255
        mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

        # Mengubah warna masker
        colored_mask = np.zeros((H, W, 3), dtype=np.uint8)
        colored_mask[mask > 0] = [0, 0, 255]  # Ganti warna di sini, misalnya [0, 255, 0] menjadi [255, 0, 0] untuk warna biru

        all_masks.append(colored_mask)

# Menggabungkan semua masker ke dalam satu gambar
combined_mask = np.zeros((H, W, 3), dtype=np.uint8)
for mask in all_masks:
    combined_mask = cv2.bitwise_or(combined_mask, mask)

# Menggabungkan gambar asli dengan hasil segmentasi
result_image = cv2.bitwise_or(img, combined_mask)

# Menyimpan gambar yang telah digabungkan
cv2.imwrite('./output_combined_colored.png', result_image)



0: 640x640 1 duck, 278.5ms
Speed: 15.6ms preprocess, 278.5ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)


True